# Importacoes

In [ ]:
import tensorflow as tf


In [ ]:
print(tf.__version__)


In [ ]:
import requests
import pandas as pd
import numpy as np


# Conectando com servidor

In [ ]:
diaInicio = '20230301'
diaFinal = '20230531'
idGrupo = '15'
idRegra = '135'
idTipo = 'null'
url = 'http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoSaidas/listaParaTreino?diaNumInicio={}&diaNumFinal={}&idGrupoAcao={}&idRegraProjecao={}&idTipoExemplo={}'.format(diaInicio,diaFinal,idGrupo,idRegra,idTipo)


In [ ]:
print(url)


In [ ]:
r = requests.get(url)
j = r.json()
df = pd.DataFrame.from_dict(j)


In [ ]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')


In [ ]:
dfTratadoX = df.apply(trataLinhaX, axis='columns')
X = np.stack(dfTratadoX.values)
X.shape


In [ ]:
Y = np.stack(df['campoY'].values)
Y = Y.reshape(-1,1)
Y.shape


# Dados Validacao

In [ ]:
diaInicio = '20230531'
diaFinal = '20230531'
idGrupo = '15'
idRegra = '135'
idTipo = 'null'
urlValida = 'http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoSaidas/listaParaTreino?diaNumInicio={}&diaNumFinal={}&idGrupoAcao={}&idRegraProjecao={}&idTipoExemplo={}'.format(diaInicio,diaFinal,idGrupo,idRegra,idTipo)


In [ ]:
print(urlValida)


In [ ]:
r = requests.get(urlValida)
j = r.json()
dfValida = pd.DataFrame.from_dict(j)


In [ ]:
dfValida.head()


In [ ]:
dfTratadoValidaX = dfValida.apply(trataLinhaX, axis='columns')
XValida = np.stack(dfTratadoValidaX.values)
XValida.shape


In [ ]:
YValida = np.stack(dfValida['campoY'].values)
YValida = YValida.reshape(-1,1)
YValida.shape


# Minha Rede

## Estrutra

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(units=64, activation='tanh', kernel_initializer=tf.keras.initializers.HeUniform(), return_sequences=True, input_shape=(300,1)),
    tf.keras.layers.LSTM(units=32, activation='tanh', kernel_initializer=tf.keras.initializers.HeUniform(), return_sequences=True),
    tf.keras.layers.LSTM(units=16, activation='tanh', kernel_initializer=tf.keras.initializers.HeUniform()),
    tf.keras.layers.Dense(units=1, activation='sigmoid', kernel_initializer=tf.keras.initializers.HeUniform())
])
model.summary()


## Compilacao

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy',  metrics=['accuracy'])


## Fit - Treino

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=4)
X2 = X.T
X2 = X2[np.newaxis,...]
history = model.fit(x=X2.T,y=Y,epochs=80,verbose=2,callbacks=[callback])



## Salva Rede

In [ ]:
path='pesos/treino_rede1037.h5'
model.save(path)


# Enviando Rede

In [ ]:
!pip install paramiko


In [ ]:
import os
import paramiko


In [ ]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect("191.252.219.136", username="root", password="Digicom2004$")
sftp = ssh.open_sftp()
sftp.put("pesos/treino_rede1037.h5", "/root/executa-rede/redes/treino_rede1037.h5")
sftp.close()
ssh.close()


# Registra Treinamento

In [ ]:
#import requests
url = 'http://vps-40d69db1.vps.ovh.ca:23004/api/TreinoRedes/executouTreinamento'
myobj = {'id': 1037}
x = requests.post(url, json = myobj)
print(x.text)
